In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv("USvideos.csv")

In [3]:
preprocessed_data = data.drop(['description', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'publish_time', 'channel_title', 'trending_date', 'video_id', 'tags', 'title'], axis=1)

In [4]:
preprocessed_data.head()

,category_id,views,likes,dislikes,comment_count
0,22,748374,57527,2966,15954
1,24,2418783,97185,6146,12703
2,23,3191434,146033,5339,8181
3,24,343168,10172,666,2146
4,24,2095731,132235,1989,17518


In [5]:
X = preprocessed_data.drop('category_id', axis=1)
y = preprocessed_data['category_id']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
scaling = MinMaxScaler(feature_range=(0, 1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [8]:
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=20, random_state=33)
svc = SVC(kernel='rbf', C=1, decision_function_shape='ovr', gamma=1000)
svc.fit(X_train, y_train)
#model.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1000, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
y_pred = svc.predict(X_test)

In [12]:
accuracy_score(y_test, y_pred)

0.3081807081807082